In [2]:
%pip install -r requirements.txt -q


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import urllib.request
import pandas as pd
from dotenv import load_dotenv
from pinecone import Pinecone

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone as PineconeStore
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings

from langchain_openai import ChatOpenAI  
from langchain.chains import RetrievalQA  

/Users/mira/VScode/chatbot/.venv/lib/python3.12/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [4]:
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = openai_api_key

pinecone_api_key = os.getenv('PINECONE_API_KEY')
os.environ["PINECONE_API_KEY"] = pinecone_api_key

In [5]:
df = pd.read_csv('/Users/mira/VScode/chatbot/data/my_profile.csv',header=None,names=['text','Category'])

# OpenAIEmbeddings インスタンスを作成
embeddings = OpenAIEmbeddings()

# 'text_column' の各セルをベクトル化してリストに保存
df['vectorized'] = df['text'].apply(lambda x: embeddings.embed_query(x))

# ベクトルリストをデータフレームに展開（各要素を別々のカラムに）
vectorized_df = pd.DataFrame(df['vectorized'].tolist(), index=df.index)

# 元のDataFrameとベクトル化したDataFrameを結合
output_df = pd.concat([df.drop(columns=['vectorized']), vectorized_df], axis=1)

In [6]:
pc = Pinecone(api_key=pinecone_api_key)
pinecone_index = pc.Index("sample-db")
pc.list_indexes()

{'indexes': [{'deletion_protection': 'disabled',
              'dimension': 1536,
              'host': 'sample-db-rdebc4f.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'sample-db',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [7]:
original_texts = output_df.iloc[:, 0]
original_category = output_df.iloc[:, 1]
vectorized_data_only = output_df.iloc[:, 2:]

In [8]:
for i in range(len(vectorized_data_only)):
    pinecone_index.upsert(
        vectors = [
            {
                'id': str(i+1),
                'values': vectorized_data_only.T[i],
                'metadata': {"text": original_texts[i], "Category": original_category[i]}
            }
        ]
    )

In [9]:
pc.list_indexes()

{'indexes': [{'deletion_protection': 'disabled',
              'dimension': 1536,
              'host': 'sample-db-rdebc4f.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'sample-db',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [10]:
env="aws"
index_name = "sample-db"


embeddings = OpenAIEmbeddings()

#vectorstore = PineconeStore.from_documents(index_name, embeddings,"text")
vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings,pinecone_api_key=pinecone_api_key)
#retriever = PineconeVectorStore(search_type="similarity", search_kwargs={"k": 5})

In [11]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

# プロンプトテンプレートの定義
prompt_template = """Use the following pieces of context: {context} 
感情が伝わるように「！」など感情表現豊かにしてください。全てに「！」をつけないでください、適切なタイミングで使うようにして。また一問一答のように質問にだけ応えて。: {question}"""
PROMPT = PromptTemplate(
    template=prompt_template, 
    input_variables=["context", "question"]
)

# completion llm  
llm = ChatOpenAI(  
    model_name='gpt-4o-mini',  
    temperature=0,
)  

# RetrievalQAの設定
qa = RetrievalQA.from_chain_type(  
    llm=llm,  
    chain_type="stuff",  
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs={"prompt": PROMPT, "document_variable_name": "context"} 
)  

# インタラクティブな会話ループ
context = ""  # 初期文脈
while True:
    q = input("あなたの質問: ")  # ユーザーからの質問を取得
    result = qa.invoke({"query": q, "context": context})  # 質問を実行
    print("回答:", result['result'])  # モデルの応答を表示
    
    # 次の質問のために文脈を更新
    context += f"\n質問: {q}\n回答: {result['result']}"
    
    # 終了条���
    if q.lower() in ["exit", "終了", "やめる"]:
        break


回答: 趣味は読書、散歩、プログラミングです！特にプログラミングは、最初は就活のために始めたんですが、今ではすっかり楽しくなって、自分のもう一つの趣味になりました！エディタと向き合う時間が増えて、毎日新しいことを学んでいます！
回答: 主にPython、Java、JavaScript（TypeScript）を使用しています！特に機械学習のプロジェクトではPythonが多いですね。
回答: はい、機械学習系のインターンが多いです！特に、Zigexnでの長期インターンシップやAlmondoでのLLM開発のプロジェクトを通じて、実践的な経験を積んでいます！
回答: Zigexnでの長期インターンシップでは、機械学習エンジニアとしてさまざまなプロジェクトに取り組んでいます！具体的には、データ分析やモデルの構築、そして実際のビジネス課題に対する機械学習の適用を行っています。特に、自分専用のChatBotの作成を通じて技術力を高めることに力を入れています！このプロジェクトは、自分の技術力を外にアピールする良い機会にもなっています！
回答: はい！AlmondoでのLLM開発のプロジェクトでは、プロジェクトマネージャーとしてチームをリードし、効率的な開発を進めています！さまざまな技術的課題に取り組みながら、最適なソリューションを見つけることに注力しています。チームメンバーとのコミュニケーションも大切にしていて、みんなで協力しながら進めています！
回答: やめる理由は何ですか？


In [12]:
q='長期インターンシップで何をしていますか'
result = qa.invoke({"query": q})
display("Answer:", result['result'])

'Answer:'

'長期インターンシップでは、機械学習エンジニアとしてさまざまなプロジェクトに取り組んでいます！具体的には、データの前処理やモデルの構築、評価などを行い、実際のビジネスに役立つ機械学習ソリューションを開発しています。日々新しい技術や知識を学びながら、チームと協力して成果を出すことにやりがいを感じています！'